In [1]:
## LIBRARIES
# ---

# Selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains

# Options driver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

# Dataframes
import pandas as pd
import itertools
import os

# Simulating human behavior
import time
from time import sleep
import random

# Clear data
import unidecode

# Json files
import json
import re
import numpy as np
import itertools
from pandas import json_normalize

# To use explicit waits
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Download files
import urllib.request
import requests
from openpyxl import Workbook

# pytesseract
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [2]:
def scraper_SUNEDU( df ):
    
    options = Options()
    service = Service( ChromeDriverManager().install( ) )
    driver  = webdriver.Chrome( service = service )
    driver.maximize_window()

    url     = f'https://enlinea.sunedu.gob.pe/'
    driver.get( url )
    wait_60 = WebDriverWait( driver, 60 )
    wait_30 = WebDriverWait( driver, 30 )
    wait_10 = WebDriverWait( driver, 10 )
    wait_05 = WebDriverWait( driver, 5 )
    
    verificar_titulos_popup = wait_30.until( EC.element_to_be_clickable( ( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' ) ) )
    verificar_titulos_popup.click()
    
    frame_popup = wait_30.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="ifrmShowFormConstancias"]' ) ) )
    driver.switch_to.frame( frame_popup )
    
    variable_flag = False
    
    df[ 'n_grados' ] = 0
    
    for index, row in df.iterrows():
        
        dni_valor = row[ 'dni' ]
        print( f'DNI: { dni_valor }' )
        
        dni_campo = wait_60.until( EC.element_to_be_clickable( ( By.XPATH, '//*[@id="doc"]' ) ) )
        dni_campo.clear()
        dni_campo.send_keys( dni_valor )
        
        max_retries = 15

        for retry in range( max_retries ):
        
            try: 

                time.sleep( 2 )
                captcha       = wait_05.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="captchaImg"]/img' ) ) )
                captcha_image = captcha.screenshot_as_png
                
                ruta_img      = f'img/img.png'
                with open( ruta_img, 'wb' ) as f:
                    f.write( captcha_image )

                clave_valor   = pytesseract.image_to_string( Image.open( ruta_img ),
                                                             config = '-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789' ).strip()
                campo_captcha = wait_05.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="captcha"]' ) ) )
                campo_captcha.clear()
                campo_captcha.send_keys( clave_valor )   

                buscar_boton = wait_05.until( EC.element_to_be_clickable( ( By.XPATH, '//*[@id="buscar"]' ) ) )
                buscar_boton.click()
                
                print( f'Captcha: { clave_valor }' )
                
                # time.sleep( 10 )
                
                try: 
                    
                    sections        = {}
                    result_sections = wait_30.until( EC.presence_of_all_elements_located( (By.XPATH, '//*[@id="finalData"]/tr') ) )
                    n_grados        = len( result_sections )

                    for i, result in enumerate( result_sections ):
                        
                        boxes = wait_30.until( EC.presence_of_all_elements_located( ( By.XPATH, f'//*[@id="finalData"]/tr[{ i + 1 }]/td' ) ) )
                        
                        for j, box in enumerate( boxes ):
                            
                            boxes_names_list = { 0: 'datos', 1: 'grado_fecha', 2: 'institucion', 3: 'otro' }
                            box_name = boxes_names_list.get( j )   
                            text     = box.text
                            sections[ f'{ box_name }_{ i }' ] = text

                    row = { 'dni': dni_valor }
                    row.update( sections )
                    df.loc[ index, ['dni', *sections.keys() ] ] = row.values()
                    df.loc[ index, [ 'n_grados' ] ] = n_grados
                    
                    columnas_a_borrar = [ col for col in df.columns if  col.startswith( 'otro' ) ]
                    df                = df.drop( columns = columnas_a_borrar )
                    
                    print( f'Retry: { retry }' )
                    print( f'Se saltó el captcha' )
                    print( 'Éxito en extraer datos' )

                    driver.switch_to.default_content()
                    driver.refresh()
                    
                    verificar_titulos_popup = wait_30.until( EC.element_to_be_clickable( ( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' ) ) )
                    verificar_titulos_popup.click()
                    
                    frame_popup = wait_30.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="ifrmShowFormConstancias"]') ) )
                    driver.switch_to.frame( frame_popup )    
                    
                    print( 'Pasamos a la siguiente observación' )
                    print( '\n' )

                    break                    
                
                except:   
                
                    error_popup        = wait_30.until( EC.element_to_be_clickable( ( By.XPATH, '//*[@id="frmError"]' ) ) )
                    class_error_popup  = error_popup.get_attribute( 'class' )
                    caso_no_resultados = 'No se encontraron resultados con este número de DNI'
                    error_msj          = wait_05.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="frmError_Body"]/p' ) ) ).text
                    
                    if error_msj.startswith( caso_no_resultados ) == False:
                        
                        raise Exception( 'Continuar hacia el bloque except' )
                    
                    else:
                        
                        print( f'Retry: { retry }' )
                        print( f'Se saltó el captcha' )
                        print( f'No se encontraron resultados' )
                        print( 'Pasamos a la siguiente observación' )
                        print( '\n' )   
                        
                        driver.switch_to.default_content()
                        driver.refresh()
                        
                        verificar_titulos_popup = wait_30.until( EC.element_to_be_clickable( ( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' ) ) )
                        verificar_titulos_popup.click()
                        
                        frame_popup = wait_30.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="ifrmShowFormConstancias"]' ) ) )
                        driver.switch_to.frame( frame_popup )    
                        
                        variable_flag = True                        
                        
                        break
                    
                if variable_flag:
                    
                    break
                
            except Exception as e:

                print( f'Retry: { retry }' )
                print( f'Error en la ejecución' )
                print( f'Intentando de nuevo después de 5 segundos...' )
                
                try: 
                    
                    boton_x = wait_30.until( EC.element_to_be_clickable( ( By.XPATH, '//*[@id="closeModalError"]' ) ) )
                    boton_x.click()
                    
                    print( f'Click en botón X' )

                    continue
                    
                except:
                    
                    driver.switch_to.default_content()
                    driver.refresh()

                    verificar_titulos_popup = wait_30.until( EC.element_to_be_clickable( ( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' ) ) )
                    verificar_titulos_popup.click()

                    frame_popup = wait_30.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="ifrmShowFormConstancias"]' ) ) )
                    driver.switch_to.frame( frame_popup )   

                    dni_campo = wait_60.until( EC.element_to_be_clickable( ( By.XPATH, '//*[@id="doc"]' ) ) )
                    dni_campo.clear()
                    dni_campo.send_keys( dni_valor )
                    
                    print( f'Se reinicia el navegador' )
                    
                    continue
                    
    driver.quit()
        
    return df

In [3]:
data = pd.read_excel( 'input/data__.xlsx', dtype = { 'dni': str } )
data

,dni
0,46432891
1,32224811


In [4]:
updated_data = scraper_SUNEDU( data )

DNI: 46432891
Captcha: WS5BU5
Retry: 0
Error en la ejecución
Intentando de nuevo después de 5 segundos...
Click en botón X
Captcha: GT4X2
Retry: 1
Se saltó el captcha
No se encontraron resultados
Pasamos a la siguiente observación


DNI: 32224811
Captcha: 68RZN
Retry: 0
Se saltó el captcha
Éxito en extraer datos
Pasamos a la siguiente observación




In [9]:
updated_data = scraper_SUNEDU( data )

DNI: 46432891
Retry: 14
Error en la ejecución
Intentando de nuevo después de 5 segundos...
Click en botón X
DNI: 70313538
Retry: 14
Error en la ejecución
Intentando de nuevo después de 5 segundos...


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.124)
Stacktrace:
	GetHandleVerifier [0x006E72A3+45731]
	(No symbol) [0x00672D51]
	(No symbol) [0x0056880D]
	(No symbol) [0x0054F75E]
	(No symbol) [0x005BC11B]
	(No symbol) [0x005CB2D3]
	(No symbol) [0x005B7DD6]
	(No symbol) [0x005931F6]
	(No symbol) [0x0059439D]
	GetHandleVerifier [0x009F0716+3229462]
	GetHandleVerifier [0x00A384C8+3523784]
	GetHandleVerifier [0x00A3214C+3498316]
	GetHandleVerifier [0x00771680+611968]
	(No symbol) [0x0067CCCC]
	(No symbol) [0x00678DF8]
	(No symbol) [0x00678F1D]
	(No symbol) [0x0066B2C7]
	BaseThreadInitThunk [0x7760FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D07C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77D07C3E+238]


In [7]:
updated_data

,dni,n_grados,datos_0,grado_fecha_0,institucion_0,datos_1,grado_fecha_1,institucion_1,datos_2,grado_fecha_2,institucion_2,datos_3,grado_fecha_3,institucion_3
0,46432891,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,70313538,1,"CALDAS VELASQUEZ, JOSUE DANIEL\nDNI 70313538",Bachiller en Ciencias Sociales con Mención en ...,PONTIFICIA UNIVERSIDAD CATÓLICA DEL PERÚ\nPERU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,71696567,1,"CASTILLO ESPINOZA, DARLA APRIL\nDNI 71696567",Bachiller en Ciencias Sociales con Mención en ...,PONTIFICIA UNIVERSIDAD CATÓLICA DEL PERÚ\nPERU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72675329,1,"CUTIPA APAZA, ROBERTO CARLOS\nDNI 72675329",Bachiller en Ciencias Sociales con Mención en ...,PONTIFICIA UNIVERSIDAD CATÓLICA DEL PERÚ\nPERU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,32220818,2,"VELASQUEZ TREVEJO, SANTOS HERMELINDA\nDNI 3222...",LICENCIADA EN EDUCACION PRIMARIA\n\nFecha de d...,UNIVERSIDAD PRIVADA DE SAN PEDRO\nPERU,"VELASQUEZ TREVEJO, SANTOS HERMELINDA\nDNI 3222...",BACHILLER EN EDUCACION\n\nFecha de diploma: 01...,UNIVERSIDAD PRIVADA DE SAN PEDRO\nPERU,NaN,NaN,NaN,NaN,NaN,NaN
5,32224811,4,"CALDAS CAÑARI, ROMEL DAVID\nDNI 32224811",LICENCIADO EN CIENCIAS DE LA EDUCACION\n\nFech...,UNIVERSIDAD NACIONAL DE EDUCACIÓN ENRIQUE GUZM...,"CALDAS CAÑARI, ROMEL DAVID\nDNI 32224811",LICENCIADO EN EDUCACION. ESPECIALIDAD:\n\nFech...,UNIVERSIDAD NACIONAL DE EDUCACIÓN ENRIQUE GUZM...,"CALDAS CAÑARI, ROMEL DAVID\nDNI 32224811",BACHILLER EN CIENCIAS DE LA EDUCACION\n\nFecha...,UNIVERSIDAD NACIONAL DE EDUCACIÓN ENRIQUE GUZM...,"CALDAS CAÑARI, ROMEL DAVID\nDNI 32224811",MAESTRO EN EDUCACION CON MENCION EN DOCENCIA U...,UNIVERSIDAD SAN PEDRO\nPERU
6,75947795,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,72653667,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
updated_data.to_excel( 'output/updated_data.xlsx' )